In [ ]:
# import os
# os.environ["SSL_CERT_FILE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"
# os.environ["REQUESTS_CA_BUNDLE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"

In [ ]:
import json
import pandas as pd
from tqdm.auto import tqdm
import os

/mnt/d/Travel Assistant/Musafir/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('../data/processed_data/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)


In [4]:
documents[500]

{'city': 'Seoul',
 'section': 'See',
 'subsection': 'Parks and mountains',
 'text': "Hangang Citizen's Park . Alongside the Han River through 13 districts: Gwangnaru, Jamsil, Gangdong, Ttukseom, Jamwon, Banpo, Ichon, Yeouido, Yanghwa, Mangwon, Seonyudo, Nanji, and Gangseojigu. You can see many people strolling or jogging along the trail paths, as well as in-line skaters, bicyclists, and soccer fields or basketball courts. Yeouido, Jamsil, and Ttukseom districts are especially popular because of the cruise services on the Han River.",
 'id': '9a39d77f'}

In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-09-28 14:15:36--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4350 (4.2K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   4.25K  --.-KB/s    in 0.002s  

2025-09-28 14:15:36 (2.47 MB/s) - ‘minsearch.py.2’ saved [4350/4350]



In [6]:
import minsearch

/mnt/d/Travel Assistant/Musafir/notebooks/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [7]:
df_gt = pd.read_csv('../data/result/groud-truth-retrieval.csv')

In [8]:
df_gt.head()

,id,city,question
0,f7845786,Cairo,What is the name of the oldest known pyramid i...
1,f7845786,Cairo,Which pyramid in Dahshur has an entrance to th...
2,f7845786,Cairo,What is the distinctive feature of the Bent Py...
3,f7845786,Cairo,How many pyramids are mentioned to be in the D...
4,f7845786,Cairo,What is the atmosphere around Dahshur Pyramids...


In [9]:
ground_truth = df_gt.to_dict(orient='records')


In [10]:
ground_truth[:5]

[{'id': 'f7845786',
  'city': 'Cairo',
  'question': 'What is the name of the oldest known pyramid in Dahshur?'},
 {'id': 'f7845786',
  'city': 'Cairo',
  'question': 'Which pyramid in Dahshur has an entrance to the inside?'},
 {'id': 'f7845786',
  'city': 'Cairo',
  'question': 'What is the distinctive feature of the Bent Pyramid?'},
 {'id': 'f7845786',
  'city': 'Cairo',
  'question': 'How many pyramids are mentioned to be in the Dahshur area?'},
 {'id': 'f7845786',
  'city': 'Cairo',
  'question': 'What is the atmosphere around Dahshur Pyramids like compared to Giza?'}]

In [11]:
index = minsearch.Index(
    text_fields=['section', 'text'],
    keyword_fields=['id', 'city']
)

index.fit(documents)

In [12]:
# Recall
def recall(relevance_total):
    scores = []
    for line in relevance_total:
        total_relevant = sum(line)
        if total_relevant == 0:
            scores.append(0)
        else:
            retrieved_relevant = sum(line)
            scores.append(retrieved_relevant / total_relevant)
    return sum(scores) / len(scores)

In [13]:
#Hit Rate (HR) or Recall at k
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [14]:
# Mean Reciprocal Rank (MRR)
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [15]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results


In [16]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'Recall': recall(relevance_total),
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }
        

In [17]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:09<00:00, 277.06it/s]


{'Recall': 0.8552486187845304,
 'hit_rate': 0.8552486187845304,
 'mrr': 0.7135870677307133}

In [18]:
def minsearch_search_filter(query, city):
    boost = {'text': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'city': city},
        boost_dict=boost,
        num_results=5
    )

    return results
    

In [19]:
evaluate(ground_truth, lambda q: minsearch_search_filter(q['question'], q['city']))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:10<00:00, 268.85it/s]


{'Recall': 0.8589318600368324,
 'hit_rate': 0.8589318600368324,
 'mrr': 0.7564702271332095}

### Baseline Minsearch
{'Recall': 0.8552486187845304,

 'hit_rate': 0.8552486187845304,
 
 'mrr': 0.7135870677307133}

## Finding the best paramters


In [20]:
df_validation = df_gt[:150]
df_test = df_gt[150:]

In [21]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [22]:
gt_val = df_validation.to_dict(orient='records')

In [23]:
param_ranges = {
 'city': (0.0, 3.0),
 'section': (0.0, 3.0),
 'text': (0.0, 3.0),
}

In [24]:
def minsearch_search_par(query, boost=None):
    
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [25]:
def objective(boost_params):
    def search_function(q):
        return minsearch_search_par(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [26]:
simple_optimize(param_ranges, objective, n_iterations=40)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 282.21it/s]


({'city': 2.9131687940774844,
  'section': 0.17011655692307248,
  'text': 2.930460033292322},
 0.7959894179894179)

In [28]:
# Apply the boost params in the whole dataset
def minsearh_improved(query):
    
    boost = {
          'city': 2.91,
          'section': 0.17,
          'text': 2.93,
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results


In [29]:
evaluate(ground_truth, lambda q: minsearh_improved(q['question']))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:09<00:00, 280.24it/s]


{'Recall': 0.8718232044198895,
 'hit_rate': 0.8718232044198895,
 'mrr': 0.7258835394194502}

### minsearch with filter 

{'Recall': 0.8589318600368324,

 'hit_rate': 0.8589318600368324,
 
 'mrr': 0.7564702271332095}


### Baseline Minsearch
{'Recall': 0.8552486187845304,

 'hit_rate': 0.8552486187845304,
 
 'mrr': 0.7135870677307133}
 

### If the travel assistant is Q&A style (users ask short questions, expect the first result to be correct quickly), go with minsearch with filter → best MRR.

## RAG Flow 

In [35]:
minsearch_search_filter("Can I find halal food?", 'London')


[{'city': 'London',
  'section': 'Eat',
  'subsection': 'Religious',
  'text': 'Due to the mix of cultures and religions, many London restaurants cater well for religious dietary requirements. The most common signs are for Halal and Kosher meat, from burger joints to nice restaurants. There are lots of Halal restaurants and shops all over London including Whitechapel Rd and Brick Lane in the East End , Bayswater, Edgware Rd and Paddington and in many parts of north London . There are plenty of Kosher restaurants in Golders Green , Edgware and Stamford Hill along with some central delis such as on Charing Cross Road.',
  'id': '70718ca8'},
 {'city': 'London',
  'section': 'Eat',
  'subsection': 'Restaurant areas',
  'text': "As one of the world's most cosmopolitan cities, you can find restaurants serving food cuisine from nearly every country, some of it as good as, if not better than in the countries of origin. Indian food in London is especially famous and there is hardly a district w

In [64]:
def build_prompt(query, search_results):
    prompt_template = """
You're a travel assistant. Answer the QUESTION based on the CONTEXT from our travel database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    entry_template = """
City: {city}
Section: {section}
Subsection: {subsection}
Text: {text}
""".strip()

    context = ""
    for doc in search_results:
        context += entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context.strip())
    return prompt


In [65]:
from mistralai import Mistral
from mistralai.models import UserMessage
import os
from dotenv import load_dotenv

In [66]:
# loads variables from .env
load_dotenv()  

True

In [67]:
api_key = os.getenv("API_KEY")

In [68]:
llm_client = Mistral(api_key = api_key)

In [69]:
def llm(prompt):
    response = llm_client.chat.complete(
        model= "ministral-8b-latest",
        messages=[UserMessage(content=prompt)],
    )

    return response.choices[0].message.content

In [70]:
def rag(query, city):
    search_results = minsearch_search_filter(query=query, city=city)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [75]:
answer = rag("Plan 3-day itinerary for Rome?", 'Roma')
print(answer)

Sure, here's a 3-day itinerary for Rome based on the provided context:

**Day 1: Ancient Rome**
- Morning:
  - Start at the Colosseum. Explore the ancient amphitheater and its history.
  - Walk to the Roman Forum, the political and commercial center of the Roman Republic.
- Afternoon:
  - Visit the Palatine Hill, one of the Seven Hills of Rome.
  - Head to the Circus Maximus, the ancient chariot-racing stadium.
- Evening:
  - Enjoy dinner at a local trattoria in the Trastevere neighborhood.

**Day 2: Vatican City**
- Morning:
  - Visit the Vatican Museums, including the Sistine Chapel.
  - Explore St. Peter's Basilica and climb the dome for a panoramic view of Rome.
- Afternoon:
  - Visit the Vatican Gardens and the Vatican Library.
- Evening:
  - Enjoy a leisurely dinner at a restaurant near the Vatican.

**Day 3: Historic Center and Modern Rome**
- Morning:
  - Visit the Pantheon, a former Roman temple, now a church.
  - Walk to Piazza Navona and see Bernini's Fountain of the Four Ri

### Use minsearch with best paramters

In [85]:
def rag_improved(query):
    search_results = minsearh_improved(query=query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [86]:
answer = rag_improved("How I can Visit pyrimads in Cairo?")
print(answer)

To visit the Pyramids in Cairo, you can follow these steps:

1. **Location**: The Pyramids of Giza are located on the Giza Plateau, about 15 kilometers southwest of Cairo's city center.

2. **Transportation**:
   - **Taxi**: You can take a taxi from downtown Cairo. The journey should take around 30-45 minutes, depending on traffic.
   - **Bus**: There are public buses that go to the Pyramids. You can take bus number 355 from downtown Cairo.
   - **Organized Tour**: Many tour operators offer day trips to the Pyramids from Cairo.

3. **Entrance**: The entrance fee for the Pyramids is 200 Egyptian Pounds (EGP) for adults and 100 EGP for students. Children under 12 enter for free.

4. **Opening Hours**: The Pyramids are open from 8:00 AM to 5:00 PM daily.
